In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn
pd.options.mode.chained_assignment = None  # default='warn'


In [3]:
df = pd.read_parquet("escooter_history_2022.parquet")

In [4]:
df_33 = df.groupby(pd.Grouper(key="datetime",freq="D")).agg({"holiday":"mean","workingday":"mean"}).reset_index()

In [5]:
df_33.head()

,datetime,holiday,workingday
0,2020-01-04,0.0,0.0
1,2020-01-05,0.0,0.0
2,2020-01-06,0.0,1.0
3,2020-01-07,0.0,1.0
4,2020-01-08,0.0,1.0


In [6]:
df_round = df_33.loc[(~df_33.workingday.isin([1,0]) | ~df_33.holiday.isin([0,1]))]
df_round.head()

,datetime,holiday,workingday
28,2020-02-01,0.0,0.000950
56,2020-02-29,0.0,0.001797
58,2020-03-02,0.0,0.997782
65,2020-03-09,0.0,0.998101
119,2020-05-02,0.0,0.000483


In [7]:
df_round = df_33.loc[(~df_33.workingday.isin([1,0]) | ~df_33.holiday.isin([0,1]))].round(1).reset_index(drop=True)
df_round.head()

,datetime,holiday,workingday
0,2020-02-01,0.0,0.0
1,2020-02-29,0.0,0.0
2,2020-03-02,0.0,1.0
3,2020-03-09,0.0,1.0
4,2020-05-02,0.0,0.0


In [8]:
df_round.count()

datetime      41
holiday       41
workingday    41
dtype: int64

In [9]:
df.loc[df.datetime.dt.date.isin(df_round.datetime.dt.date)][["datetime", "holiday", "workingday"]]

,datetime,holiday,workingday
34320,2020-02-01 00:00:25,0.0,1.0
34321,2020-02-01 00:01:11,0.0,1.0
34322,2020-02-01 00:01:21,0.0,0.0
34323,2020-02-01 00:02:18,0.0,0.0
34324,2020-02-01 00:02:24,0.0,0.0
...,...,...,...
3755410,2022-01-03 23:58:10,0.0,1.0
3755411,2022-01-03 23:58:28,0.0,1.0
3755412,2022-01-03 23:58:46,0.0,1.0
3755413,2022-01-03 23:59:16,0.0,1.0


In [11]:
for date in df_round.datetime.dt.date:

  if df.loc[df.datetime.dt.date == date] and df_round.loc[df_round.datetime == date].workingday == 1 and df.loc[df.datetime.dt.date == date].workingday != 1:
    df.replace({0 : 1})
  elif df.loc[df.datetime.dt.date == date] and df_round.loc[df_round.datetime == date].workingday != 1 and df.loc[df.datetime.dt.date == date].workingday == 1:
    df.replace({1 : 0})

  elif df.loc[df.datetime.dt.date == date] and df_round.loc[df_round.datetime == date].holiday == 1 and df.loc[df.datetime.dt.date == date].holiday != 1:
    df.replace({0 : 1})
  elif df.loc[df.datetime.dt.date == date] and df_round.loc[df_round.datetime == date].holiday != 1 and df.loc[df.datetime.dt.date == date].holiday == 1:
    df.replace({1 : 0})


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [12]:
df["workingday"] = np.where(df.loc[df.datetime.dt.date.isin(df_round.datetime.dt.date)].workingday == 1, 1, df.workingday)

ValueError: operands could not be broadcast together with shapes (265468,) () (3755415,) 

In [ ]:
df.loc[(~df.workingday.isin([1,0]) | ~df.holiday.isin([0,1]))]